In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import statsmodels.api as sm

In [2]:
data = pd.read_csv('data/train.csv')
census = pd.read_csv('data/census_starter.csv')

In [3]:
test_data = pd.read_csv('data/test.csv')

In [4]:
data['first_day_of_month'] = pd.to_datetime(data['first_day_of_month'])
test_data['first_day_of_month'] = pd.to_datetime(test_data['first_day_of_month'])

In [5]:
row_id = test_data['row_id']
test_data = test_data.drop('row_id', axis=1)

In [8]:
data.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [9]:
# data selection

X = data[['cfips','first_day_of_month']]
y = data['microbusiness_density']


In [10]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [11]:
# save metrics to dataframe
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
metrics = pd.DataFrame(columns=['model', 'rmse', 'mae', 'r2'])

def save_metrics(model, y_test, y_pred):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    metrics.loc[len(metrics)] = [model, rmse, mae, r2]

In [12]:
# auto regressive model
from statsmodels.tsa.ar_model import AutoReg

auto_reg = AutoReg(y_train, lags=1)
auto_reg_fit = auto_reg.fit()
y_pred = auto_reg_fit.predict(start=len(X_train), end=len(X_train)+len(X_test)-1, dynamic=False)
save_metrics('auto regressive', y_test, y_pred)



c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


In [15]:
# ARIMA
from statsmodels.tsa.arima.model import ARIMA

arima = ARIMA(y_train, order=(1,1,1))
arima_fit = arima.fit()
y_pred = arima_fit.predict(start=len(X_train), end=len(X_train)+len(X_test)-1, dynamic=False)
save_metrics('ARIMA', y_test, y_pred)

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [17]:
#SARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarima = SARIMAX(y_train, order=(1,1,1), seasonal_order=(1,1,1,12))
sarima_fit = sarima.fit()
y_pred = sarima_fit.predict(start=len(X_train), end=len(X_train)+len(X_test)-1, dynamic=False)
save_metrics('SARIMA', y_test, y_pred)

c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [18]:
metrics

,model,rmse,mae,r2
0,auto regressive,4.677435,2.449337,-0.000078
1,ARIMA,4.677462,2.450504,-0.000090
2,SARIMA,4.678890,2.406658,-0.000700
